In [1]:
pip install fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=88eb11f30291c31e9adef21a70016236819539e4709332d3d5f6595b0511aebe
  Stored in directory: /root/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built fire


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.0 MB/s eta 0:00:00


In [3]:
###############################
##### importing libraries #####
###############################
import json 
import os
import random
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset   
torch.backends.cudnn.benchmark=True

import pyarrow.parquet as pq
import pandas as pd
import random
import fire
import logging
import os
import csv

from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import math
from statistics import mean
import matplotlib.pyplot as plt
import pickle

In [89]:
##### Hyperparameters for federated learning #########
num_clients = 20
num_selected = 5
num_rounds = 100
epochs = 5
batch_size = 16
datapath='/content/drive/MyDrive/Reserach_Notebooks/data/'
model_name="gpt2"
tokenizer_name="gpt2"
device= 'cuda'
epochs=5
lr=0.001
warmup_steps=5000
max_seq_len=128
train_count=num_clients*1000
output_dir=""
save_model_on_epoch=False
ckpt_path="/content/drive/MyDrive/Reserach_Notebooks/ckpt/"
misc_path="/content/drive/MyDrive/Reserach_Notebooks/misc/"
#np.random.seed(112)
temp=0.5
ctrl_code="<|startoftext|>"

In [93]:

class DataComposer(torch.utils.data.Dataset):
    
    def __init__(self, control_code,texts, truncate=False, max_length=768):

        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.texts = []
        
        
        for row in texts:
            self.texts.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
                
        if truncate:
            self.texts = self.texts[:train_count]
        self.text_count = len(self.texts)
        
    def __len__(self):
        return self.text_count

    def __getitem__(self, item):
        return self.texts[item]

#Pack Tensors

In [50]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

#Read Data

In [90]:
#Dataset specific tasks
with open(datapath+'imdb/all_texts.txt') as file:
  all_texts=file.read()

all_words=all_texts.split()
text_raw=[]
frm=0
to=max_seq_len
for i in range(train_count):
  temp=' '.join(all_words[frm:to])
  text_raw.append(temp)
  frm+=max_seq_len
  to+=max_seq_len

In [91]:
# Dividing the training data into num_clients, with each client having equal number of images
def make_client_dataset(text_raw):

  splits= torch.utils.data.random_split(text_raw, [int(len(text_raw) / num_clients) for _ in range(num_clients)])
  client_data=[]
  for s in splits:
    temp=[]
    for t in s:
      temp+=[t]
    client_data+=[temp]
  
  return client_data

traindata_split = make_client_dataset(text_raw)

In [94]:
# Creating a pytorch loader for a Deep Learning model
train_loaders = [torch.utils.data.DataLoader(DataComposer(ctrl_code, x, truncate=True), batch_size=1, shuffle=True) for x in traindata_split]

In [95]:
def client_update(client_model, optimizer, train_dataloader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    client_model = client_model.to(device)
    client_model.train()

    # scheduler = get_linear_schedule_with_warmup(
    #     optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    # )

    #accumulating_batch_count = 0
    input_tensor = None
    
    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        for idx, entry in enumerate(train_dataloader):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, max_seq_len)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = client_model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            optimizer.step()
            #scheduler.step()
            optimizer.zero_grad()
            client_model.zero_grad()

            #accumulating_batch_count += 1
            input_tensor = None
    return loss.item()

In [96]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [97]:
############################################
#### Initializing models and optimizer  ####
############################################

#### global model ##########
gen_model=GPT2LMHeadModel.from_pretrained(model_name)

### Resume from last ckpt
#gen_model.load_state_dict(torch.load(ckpt_path+'global_50.pt'))

global_model = gen_model

############## client models ##############
client_models = [ gen_model for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model 

############### optimizers ################
opt = [AdamW(model.parameters(), lr=lr) for model in client_models]

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [64]:
text_raw=None

## FL Training

In [ ]:
for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    for i in tqdm(range(num_selected)):
        client_update(client_models[i], opt[i], train_loaders[client_idx[i]], epoch=epochs)
    
    # server aggregate
    server_aggregate(global_model, client_models)

    print('after round ',r+1, 'saving global ckpt', 'global_'+str(r+1)+'.pt')
    
    torch.save(global_model.state_dict(), ckpt_path+'part_'+str(r+1)+'.pt')

  0%|          | 0/5 [00:00<?, ?it/s]

Training epoch 0
Training epoch 1
Training epoch 2
Training epoch 3


In [ ]:
#get perplexity of a text sample
def get_ppl(
    m_name,
    tokenizer,
    sample
):
    global_model.load_state_dict(torch.load(ckpt_path+m_name,map_location='cpu'))
    model=global_model
    model.eval()

    with torch.no_grad():
      generated = torch.tensor(tokenizer.encode(sample)).unsqueeze(0)
      outputs = model(generated, labels=generated)
      loss= outputs[0]
      ppl=torch.exp(loss)

      return ppl.item()

In [ ]:
## Generating texts with seed

def generate_text(
    m_name,
    tokenizer,
    prompt,
    entry_length=5,
    entry_count=1,
    top_p=0.8,
    temperature=1.,
):
    global_model.load_state_dict(torch.load(ckpt_path+m_name,map_location='cpu'))
    model=global_model
    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            # Using top-p (nucleus sampling): https://github.com/huggingface/transformers/blob/master/examples/run_generation.py
            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(
                    F.softmax(sorted_logits, dim=-1), dim=-1
                )

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)

                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                generated_list.append(output_text)
    ppxls=[]
    for g in generated_list:
      ppxls.append(get_PPl(m_name, tokenizer, g))
    return generated_list, ppxls

In [ ]:
saved_model='part_26.pt'
generated_texts, pp = generate_txt(saved_model, GPT2Tokenizer.from_pretrained('gpt2'),"Gallen was born in", 3, 20)

for i in range(len(generated_texts)):
  print(generated_texts[i])
  print(pp[i])